In [ ]:
!pip install -q pyomo 

In [ ]:
from pyomo. environ import *

In [ ]:
import  numpy as np

In [ ]:
#create a model
model=ConcreteModel()

In [ ]:
N=10
M=3

In [ ]:
obj_coef_A=np.array([[20, 25, 10, 30, 12.5, 35, 47.5, 10, 20, 50],
                     [75, 65, 70, 60, 85, 52.5, 42.5, 40, 35, 40],
                     [ 5, 10, 20, 10, 2.5, 12.5, 10 ,50, 45, 10]])

In [ ]:
obj_coef_inhouse=np.array([ 45, 48, 55, 65, 54, 37.5, 35, 52.5, 47, 28])
obj_coef_inpurchase=np.array([82, 102, 150, 105, 95, 84, 88, 125, 70, 64])
coef_rhs_b=[11687.5,24437.5,6375]

In [ ]:
#The following numpy arrays will be useful for setting the lower bound and upper bounds for the variables
lower_bound_v= np.array([0,0,0,0,0,0,0,0,0,0])
upper_bound_v = np.array([np.inf]*10)
lower_bound_u= np.array([0,0,0,0,0,0,0,0,0,0])
upper_bound_u= np.array([15,16,9,17.5,12,18.5,2.5,0.5,9.5,8])


In [ ]:
# set of column indices: since we have N variables, we can take column indices to be the list [0,1,2,...,N-1]
col_indices = np.arange(N)
print(col_indices)


[0 1 2 3 4 5 6 7 8 9]


In [ ]:
model.u= Var(col_indices)

In [ ]:
model.v= Var(col_indices)


In [ ]:
# create a ConstraintList to hold multiple constraints
model.constraints = ConstraintList()

In [ ]:
for i in range(len(coef_rhs_b)):
    model.constraints.add(sum(obj_coef_A[i][j]*(model.u[j]+model.v[j]) for j in col_indices)==coef_rhs_b[i])


In [ ]:
model.objective=Objective(expr=sum((obj_coef_inpurchase[i]*model.v[i] +obj_coef_inhouse[i]*model.u[i]) for i in col_indices),sense=minimize)

In [ ]:
for i in col_indices:
    model.v[i].setlb(lower_bound_v[i])
    model.v[i].setub(upper_bound_v[i])

In [ ]:
for i in col_indices:
    model.u[i].setlb(lower_bound_u[i])
    model.u[i].setub(upper_bound_u[i])


In [ ]:
model.pprint()

3 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    u_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    v_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

2 Var Declarations
    u : Size=10, Index=u_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  15.0 : False :  True :  Reals
          1 :     0 :  None :  16.0 : False :  True :  Reals
          2 :     0 :  None :   9.0 : False :  True :  Reals
          3 :     0 :  None :  17.5 : False :  True :  Reals
          4 :     0 :  None :  12.0 : False :  True :  Reals
          5 :     0 :  None :  18.5 : False :  True :  Reals
          6 :  

In [ ]:
!apt-get install -y -qq glpk-utils

In [ ]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 28747.1875
  Upper bound: 28747.1875
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 21
  Number of nonzeros: 61
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.018364667892456055
# ----------------------------------------------------------
#   Solution Information
# ------------------------

In [ ]:
# display solution
print('\nobjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('u[',i,'] = ', model.u[i].value)
  print('v[',i,'] = ', model.v[i].value)

print('\nConstraints')
model.constraints.display()


objective =  28747.1875

Decision Variables
u[ 0 ] =  15.0
v[ 0 ] =  158.078125
u[ 1 ] =  16.0
v[ 1 ] =  0.0
u[ 2 ] =  9.0
v[ 2 ] =  0.0
u[ 3 ] =  17.5
v[ 3 ] =  0.0
u[ 4 ] =  12.0
v[ 4 ] =  0.0
u[ 5 ] =  18.5
v[ 5 ] =  0.0
u[ 6 ] =  2.5
v[ 6 ] =  0.0
u[ 7 ] =  0.5
v[ 7 ] =  0.0
u[ 8 ] =  9.5
v[ 8 ] =  74.046875
u[ 9 ] =  8.0
v[ 9 ] =  84.375

Constraints
constraints : Size=3
    Key : Lower   : Body    : Upper
      1 : 11687.5 : 11687.5 : 11687.5
      2 : 24437.5 : 24437.5 : 24437.5
      3 :  6375.0 :  6375.0 :  6375.0


 1 ans --min , 82u0 + 45.0v0 + 102u1 + 48.0v1 + 150u2 + 55.0v2 + 105u3 + 65.0v3 + 95u4 + 54.0v4 + 84u5 + 37.5v5 + 88u6 + 35.0v6 + 125u7 + 52.5v7 + 70u8 + 47.0v8 + 64u9 + 28.0v9  


 0.2(u0 + v0) + 0.25*(u1 + v1) + 0.1(u2 + v2) + 0.3(u3 + v3) + 0.125(u4 + v4) +0.35000000000000003*(u5] + v5) + 0.47500000000000003*(u6 + v6) + 0.1(u7 + v7) + 0.2(u8 + v8) + 0.5(u9 + v9) = 11687.5
 

0.75(u0 + v0) + 0.65(u1 + v1) + 0.7000000000000001(u2 + v2) + 0.6(u3 + u3) + 0.85(u4 + v4) + 0.525(u5 + v5) + 0.425(u6 + v6) + 0.4(u7 + v7) + 0.35000000000000003(u8 + v8) + 0.4(u9] + v[9])=24437.5
 
 

0.05u0 + v0+ 0.1u1 + v1) + 0.2(u2 + v2) + 0.1*(u3 + v3) + 0.025(u4 + v4) + 0.125(u5 + v5) + 0.1(u6 + v6) + 0.5(u7 + v7) + 0.45(u8 + v8) + 0.1(u9] + v9)=6373 


 ans3--the optimal solution is=28747.1875
and the dicision variale are=\\Decision Variables
u[ 0 ] =  15.0
v[ 0 ] =  158.078125
u[ 1 ] =  16.0
v[ 1 ] =  0.0
u[ 2 ] =  9.0
v[ 2 ] =  0.0
u[ 3 ] =  17.5
v[ 3 ] =  0.0
u[ 4 ] =  12.0
v[ 4 ] =  0.0
u[ 5 ] =  18.5
v[ 5 ] =  0.0
u[ 6 ] =  2.5
v[ 6 ] =  0.0
u[ 7 ] =  0.5
v[ 7 ] =  0.0
u[ 8 ] =  9.5
v[ 8 ] =  74.046875
u[ 9 ] =  8.0
v[ 9 ] =  84.375










 ans-4high quantities purchased alloy are v0,v9,v8

ans-5, yes all alloy which are stock completly utiized

ans-6, yes in the stock in abiable, that reason why we will be buy in the market and it is to costly in the market that reason not need to by in market